In [3]:
import numpy as np
import pandas as pd

In [11]:
df=pd.read_csv('Goal_Kicking_NRL 2020.csv')

#Add to dataframe
#Assumption 1: left foot kick = right foot kick

def x_cord(row):
    if row.Side == 'L':
        return -row.Width
    else:
        return row.Width
    
print(df["Width"])
df['x_cord']=df.apply(lambda row: x_cord(row), axis=1)
    
print(df)


0      19
1      16
2       6
3       3
4      18
       ..
123    17
124    23
125    15
126    33
127     4
Name: Width, Length: 128, dtype: int64
     Kick     MatchID  Rd  Cond    HomeTeam    AwayTeam  HomeSc  AwaySc  \
0       1  2020110110   1  Fine  Parramatta  Canterbury       0       0   
1       2  2020110110   1  Fine  Parramatta  Canterbury       0       2   
2       3  2020110110   1  Fine  Parramatta  Canterbury       0       2   
3       4  2020110110   1  Fine  Parramatta  Canterbury       6       2   
4       5  2020110120   1  Fine    Canberra  Gold Coast       4       0   
..    ...         ...  ..   ...         ...         ...     ...     ...   
123   124  2020110280   2  Fine  Gold Coast  Parramatta       6      24   
124   125  2020110280   2  Fine  Gold Coast  Parramatta       6      30   
125   126  2020110280   2  Fine  Gold Coast  Parramatta       6      36   
126   127  2020110280   2  Fine  Gold Coast  Parramatta       6      42   
127   128  2020110280   2 